In [1]:
import pandas as pd

nw = pd.read_csv('./new_data.csv')
nw.head()

,주문번호,수령방법,결제방법,항목,추가선택,드레싱,서비스타입,주문금액,배달팁,결제금액,주문시각,접수시각,배달시각,주소,요청사항,배달요청사항
0,4617750,매장식사,나우웨이팅 POS,목살 샐러드,NaN,발사믹,onsite,6400,0,6400,2020-08-17 18:40:23,-,-,X,NaN,-
1,4617659,포장,나우웨이팅 POS,닭가슴살 샐러드,강낭콩X,오리엔탈,onsite,5900,0,5900,2020-08-17 18:33:59,-,-,X,NaN,-
2,4617644,포장,나우웨이팅 POS,숲속 샐러드,"올리브X, 고구마 50g 추가",발사믹,onsite,6600,0,6600,2020-08-17 18:33:20,-,-,X,NaN,-
3,4617627,"매장식사, 매장식사",나우웨이팅 POS,"숲속 샐러드, 목살 샐러드",고구마(50g),"오리엔탈, 오리엔탈",onsite,13000,0,13000,2020-08-17 18:31:50,-,-,X,NaN,-
4,4617575,포장,나우웨이팅 POS,이너프 샐러드,할라피뇨 X,발사믹,onsite,4300,0,4300,2020-08-17 18:28:39,-,-,X,NaN,-


In [2]:
bm = pd.read_csv('./baemin_compiled.csv')
bm.head()

,주문번호,수령방법,결제방법,항목,추가선택,드레싱,서비스타입,주문금액,배달팁,결제금액,주문시각,접수시각,배달시각,주소,요청사항,배달요청사항
0,B0KB00H8UL,배달,바로결제,"훈제연어 샐러드, 훈제연어 샐러드",NaN,렌치,배민,18800,2000,20800,2020-01-02 17:58:18,2020-01-02 18:00:26,2020-01-02 18:40:26,전북 전주시 덕진구 금암1동 ****,NaN,[]
1,B0KC005QKF,배달,바로결제,"닭가슴살 샐러드, 훈제연어 샐러드",NaN,"오리엔탈, 렌치",배민,11900,3000,14900,2020-01-03 11:56:51,2020-01-03 11:57:13,2020-01-03 12:37:13,전북 전주시 덕진구 덕진동 ****,NaN,[]
2,B0KC00731E,배달,바로결제,"목살 샐러드, 훈제연어 샐러드",오리엔탈 드레싱,"발사믹, 렌치",배민,16300,1100,17400,2020-01-03 12:29:28,2020-01-03 12:30:13,2020-01-03 13:10:13,전북 전주시 완산구 중앙동 ****,NaN,[]
3,B0KC00AE0R,배달,바로결제,"훈제연어 샐러드, 목살 샐러드","목살 (100g), 숲속토핑 (110g)","렌치, 발사믹",배민,22000,2000,24000,2020-01-03 14:08:40,2020-01-03 14:08:56,2020-01-03 14:58:56,전북 전주시 덕진구 덕진동 ****,콩이 있다면 콩을 빼주시고 올리브 많이 넣어주세요,콩이 있다면 콩을 빼주시고 올리브 많이 넣어주세요
4,B0KC00CGQW,배달,바로결제,훈제연어 샐러드,NaN,렌치,배민,9900,3500,13400,2020-01-03 15:24:54,2020-01-03 15:25:21,2020-01-03 16:15:21,전라북도 전주시 덕진구 인후2동 ****,NaN,전화주시면 바로나갈게요


In [3]:
index1 = pd.MultiIndex.from_product([['샐러드', '메인추가'], ['목살', '닭가슴살', '숲속', '연어']])
index2 = pd.MultiIndex.from_tuples([('샐러드', '이너프'),('extra','고구마'),('extra','계란'),('드레싱','발사믹'),('드레싱','오리엔탈'),('드레싱','렌치')])
columns = ['배달의민족', '키오스크', '챗봇', '현금', '총합', '비율']

In [9]:
def fill_stats(dfs):
    df = pd.DataFrame(None, index=index1, columns=columns)
    df = df + pd.DataFrame(None, index=index2, columns=columns)
    df.fillna(0, inplace=True)
    
    for data in dfs:
        for idx,row in data.iterrows():
            if row['결제방법'] == '바로결제':
                method = '배달의민족'
            elif row['결제방법'] == '만나서결제':
                method = '현금'
            elif row['결제방법'] == '나우웨이팅 POS':
                method = '키오스크'

            # 샐러드 칼럼
            for item in row['항목'].split(', '):
                if '이너프' in item:
                    df.loc[('샐러드', '이너프'),method] += 1
                elif '목살' in item:
                    df.loc[('샐러드', '목살'),method] += 1
                elif '닭가슴살' in item:
                    df.loc[('샐러드', '닭가슴살'),method] += 1
                elif '숲속' in item:
                    df.loc[('샐러드', '숲속'),method] += 1
                elif '연어' in item:
                    df.loc[('샐러드', '연어'),method] += 1

            # 메인추가, extra 칼럼   
            if type(row['추가선택']) != float:
                for item in row['추가선택'].split(', '):
                    if '고구마' in item:
                        df.loc[('extra', '고구마'),method] += 1
                    elif '계란' in item:
                        df.loc[('extra', '계란'),method] += 1
                    elif '목살' in item:
                        df.loc[('메인추가', '목살'),method] += 1
                    elif '닭가슴살' in item:
                        df.loc[('메인추가', '닭가슴살'),method] += 1
                    elif '숲속' in item:
                        df.loc[('메인추가', '숲속'),method] += 1
                    elif '연어' in item:
                        df.loc[('메인추가', '연어'),method] += 1

            # 드레싱 칼럼
            if type(row['드레싱']) != float:
                for item in row['드레싱'].split(', '):
                    if '발사막' in item:
                        df.loc[('드레싱', '발사믹'),method] += 1
                    elif '오리엔탈' in item:
                        df.loc[('드레싱', '오리엔탈'),method] += 1
                    elif '렌치' in item:
                        df.loc[('드레싱', '렌치'),method] += 1
                
    # 총합 칼럼
    df['총합'] = df.sum(axis=1)

    # 비율 칼럼
    salads = df.loc['샐러드', '총합'].sum()
    toppings = df.loc['메인추가', '총합'].sum()
    extras = df.loc['extra', '총합'].sum()
    dressings = df.loc['드레싱', '총합'].sum()

    for idx in df.index:
        pair = (idx[0], idx[1])
        if idx[0] == '샐러드':
            df.loc[pair, '비율'] = f"{int((df.loc[pair, '총합']/salads)*100)} %"
        elif idx[0] == '드레싱':
            df.loc[pair, '비율'] = f"{int((df.loc[pair, '총합']/dressings)*100)} %"
        elif idx[0] == '메인추가':
            df.loc[pair, '비율'] = f"{int((df.loc[pair, '총합']/toppings)*100)} %"
        elif idx[0] == 'extra':
            df.loc[pair, '비율'] = f"{int((df.loc[pair, '총합']/extras)*100)} %"
    return df

In [10]:
fill_stats([bm, nw])

배달의민족  키오스크  챗봇   현금    총합    비율
extra 계란        0    59   0    0    59   3 %
      고구마    1255   310   0  201  1766  96 %
드레싱   렌치     2085   354   0  322  2761  46 %
      발사믹       0     0   0    0     0   0 %
      오리엔탈   2352   527   0  310  3189  53 %
메인추가  닭가슴살    198    36   0   51   285  22 %
      목살      262    66   0   57   385  30 %
      숲속      226    39   0   69   334  26 %
      연어      171     9   0   60   240  19 %
샐러드   닭가슴살   1395   354   0  233  1982  21 %
      목살     1687   385   0  251  2323  25 %
      숲속     1113   203   0  165  1481  16 %
      연어      991   211   0  179  1381  14 %
      이너프    1672   278   0  132  2082  22 %

In [24]:
count = 0
for idx, row in nw.iterrows():
    if type(row['추가선택']) != float:
        for item in ['추가선택']:
            if '계란' in row['추가선택']:
                print(row['추가선택'])
                count += 1
print(count)

계란
계란, 고구마(100g)
계란, 고구마(100g)
올리브 X, 할라피뇨 X, 양파 X, 계란, 고구마(50g)
스위트콘 X, 할라피뇨 X, 계란 X
숲속토핑(110g) 추가, 강낭콩 X, 계란
강낭콩 X, 계란, 고구마(50g)
계란, 고구마(100g), 목살(100g) 추가, 고구마(100g)
계란
할라피뇨 X, 계란
연어(100g) 추가, 스위트콘 X, 방울토마토 X, 양파 X, 계란 X, 아몬드 X, 목살(100g) 추가, 스위트콘 X, 강낭콩 X, 계란 X, 아몬드 X
숲속토핑(110g) 추가, 강낭콩 X, 올리브 X, 할라피뇨 X, 아몬드 X, 계란
계란
계란X
양파 X, 계란, 고구마(50g)
올리브 X, 할라피뇨 X, 계란
계란 X
강낭콩 X, 계란
강낭콩 X, 계란
숲속토핑(110g) 추가, 올리브 X, 할라피뇨 X, 계란 X
닭가슴살(100g) 추가, 계란
숲속토핑(110g) 추가, 강낭콩 X, 계란, 고구마(50g)
계란, 고구마(100g)
목살(100g) 추가, 계란, 고구마(50g), 고구마(100g), 목살(100g) 추가, 양파 X, 계란, 고구마(50g)
양파X, 계란X, 아몬드X
계란
강낭콩 X, 올리브 X, 할라피뇨 X, 계란
숲속토핑(110g) 추가, 올리브 X, 할라피뇨 X, 계란 X
계란, 강낭콩 X, 올리브 X, 할라피뇨 X
계란, 고구마(50g)
목살(100g) 추가, 강낭콩 X, 고구마(50g), 닭가슴살(100g) 추가, 계란, 계란 X, 계란, 고구마(100g)
계란, 스위트콘 X, 계란
할라피뇨 X, 계란, 스위트콘 X, 올리브 X, 할라피뇨 X, 양파 X, 계란
계란, 고구마(50g)
할라피뇨X, 계란X
스위트콘 X, 계란
계란
할라피뇨X, 계란X
계란
숲속토핑(110g) 추가, 양파X, 계란X
계란X
계란X
계란X
계란, 고구마(50g)
목살(100g) 추가, 계란
고구마(50g), 계란
할라피뇨X, 계란X
계란
계란, 스위트콘 X, 올리브 X, 할라피뇨 X, 고구마(50g)
고구마(50g), 계란
계란
